In [1]:
import numpy as np
from matplotlib import pyplot as plt

def lno_data(data):
      new_data = []
      lines = data.splitlines()
      for line in lines:
            columns = line.split()
            if len(columns)>1:
                  if not line.startswith("#"): 
                        new_data.append(columns)

      new_data = np.array(new_data)

      lno_thresh = []
      for i in range(new_data.shape[0]):
            thresh_vir = new_data[:,0][i].split(sep=',')[1]
            thresh_vir = float(thresh_vir.strip('(),'))
            lno_thresh.append(thresh_vir)

      lno_data = np.array(new_data[:,1:],dtype="float32")

      lno_thresh = np.array(lno_thresh,dtype="float32")
      # lno_thresh[-1] = 1e-10 # last thresh = 0.0
      lno_afqmc_corr = lno_data[:,0]
      lno_afqmc_err = lno_data[:,1]
      lno_afqmc_mp2_corr = lno_data[:,2]
      lno_ccsd_corr = lno_data[:,3]

      lno_mp2_cr = lno_afqmc_mp2_corr-lno_afqmc_corr
      lno_ccsd_mp2_corr = lno_ccsd_corr+lno_mp2_cr
      
      return lno_thresh,lno_afqmc_corr,lno_afqmc_mp2_corr,lno_afqmc_err,lno_ccsd_corr,lno_ccsd_mp2_corr

In [1]:
from pyscf import gto, scf, tools

aB = 0.529177
a = 2*aB
b = 3*aB
nH = 6
atoms = ""
for i in range(nH):
    atoms += f"H {i*a} 0 0 \n"

mol = gto.M(atom=atoms, basis="ccpvdz", verbose=4)
mf = scf.RHF(mol)
mf.kernel()

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#1882-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Mon Sep  8 12:26:16 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 6
[INPUT] num. electrons = 6
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00

-3.2197922297840638

In [2]:
from ad_afqmc.lno_ccsd import lno_ccsd
import os
options = {'n_eql': 4,
           'n_prop_steps': 30,
            'n_ene_blocks': 1,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 10,
            'seed': 98,
            'walker_type': 'rhf',
            'trial': 'cisd',
            'dt':0.005,
            'ad_mode':None,
            'use_gpu': False,
            }
threshs = [1e-4]
for i,thresh in enumerate(threshs):
    lno_ccsd.run_lno_ccsd_afqmc(mf,thresh,[],options,nproc=5)
    os.system(f'mv results.out results.out{i+1}')

lo_type = boys


******** <class 'pyscf.lo.boys.Boys'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.01
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 13.509926645871  delta_f= 13.5099  |g|= 6.40813  1 KF 3 Hx
macro= 2  f(x)= 13.204514088961  delta_f= -0.305413  |g|= 5.62056  1 KF 3 Hx
macro= 3  f(x)= 12.942188910556  delta_f= -0.262325  |g|= 4.90673  1 KF 3 Hx
macro= 4  f(x)= 12.726909556944  delta_f= -0.215279  |g|= 4.1223  1 KF 3 Hx
macro= 5  f(x)= 12.562012765114  delta_f= -0.164897  |g|= 3.27767  1 KF 3 Hx
macro= 6  f(x)= 12.450098649392  delta_f= -0.111914  |g|= 2.38462  1 KF 3 Hx
macro= 7  f(x)= 12.392984837749  delta_f= -0.0571138  |g|= 1.45562  1 KF 3 Hx
macro= 8  f(x)= 12.385311342905  delta_f= -0.00767349  |g|= 0.50352  1 KF 2 Hx


Overwritten attributes  ao2mo  of <class 'pyscf.cc.ccsd.CCSD'>


cycle = 3  E_corr(CCSD) = -0.0960688106321653  dE = -0.00844663388  norm(t1,t2) = 0.0196283
cycle = 4  E_corr(CCSD) = -0.101029790534989  dE = -0.0049609799  norm(t1,t2) = 0.0120476
cycle = 5  E_corr(CCSD) = -0.102236970312648  dE = -0.00120717978  norm(t1,t2) = 0.0053406
cycle = 6  E_corr(CCSD) = -0.102453533338295  dE = -0.000216563026  norm(t1,t2) = 0.00299221
cycle = 7  E_corr(CCSD) = -0.102601810550316  dE = -0.000148277212  norm(t1,t2) = 0.00162866
cycle = 8  E_corr(CCSD) = -0.102606008028482  dE = -4.19747817e-06  norm(t1,t2) = 0.000702971
cycle = 9  E_corr(CCSD) = -0.102592040139603  dE = 1.39678889e-05  norm(t1,t2) = 0.000298666
cycle = 10  E_corr(CCSD) = -0.102608281556294  dE = -1.62414167e-05  norm(t1,t2) = 0.000155108
cycle = 11  E_corr(CCSD) = -0.102612911771025  dE = -4.63021473e-06  norm(t1,t2) = 5.83988e-05
cycle = 12  E_corr(CCSD) = -0.102616928354114  dE = -4.01658309e-06  norm(t1,t2) = 3.34241e-05
cycle = 13  E_corr(CCSD) = -0.10261832966085  dE = -1.40130674e-06  n